In [ ]:
tickers = [
    "PDD",          # PDD Holdings
    "SDRL",         # Seadrill
    "SMCI",         # Super Micro Computer
    "WHD",          # Cactus
    "DHT",          # DHT Holdings
    "EG",           # Everest Group
    "NFG",          # National Fuel Gas
    "PTEN",         # Patterson-UTI Energy
    "PFS",          # Provident Financial Services
    "LYC.AX",       # Lynas Rare Earths
    "PRN.AX",       # Perenti
    "GSY.TO",       # goeasy
    "BCHN.SW",      # Burckhardt Compression
    "CFR.SW",       # Richemont
    "DOC.VI",       # DO & CO
    "MAIRE.MI",     # Maire
    "MAP.MC",       # Mapfre
    "RHM.DE",       # Rheinmetall
    "RYAAY",        # Ryanair (ADR US)
    "AEG",          # Aegon (ADR US)
    "BRNL.AS",      # Brunel International
    "COK.DE",       # Cancom
    "CBK.DE",       # Commerzbank
    "DIE.BR",       # D'Ieteren
    "FSKRS.HE",     # Fiskars
    "GTT.PA",       # GTT
    "GEST.MC",      # Gestamp
    "KSB.DE",       # KSB
    "SL.MI",        # Sanlorenzo
    "SBO.VI",       # SBO AG
    "HO.PA",        # Thales
    "FR.PA",        # Valeo
    "VCT.PA",       # Vicat
    "WAVE.PA",      # Wavestone
    "MONY.L",       # MONY Group
    "BEZ.L",        # Beazley
    "DRX.L",        # Drax
    "INCH.L",       # Inchcape
    "TBCG.L",       # TBC Bank Group
    "1908.HK",      # C&D International
    "3320.HK",      # China Resources Pharmaceutical
    "1138.HK",      # COSCO Shipping Energy Transportation
    "2367.HK",      # Giant Biogene
    "2005.HK",      # SSY Group
    "1585.HK",      # Yadea
    "RICHTER.BD",   # Richter Gedeon
    "ADMIE.AT",     # Admie Holding
    "6532.T",       # BayCurrent Consulting
    "7267.T",       # Honda
    "ELK.OL",       # Elkem
    "MOWI.OL",      # Mowi
    "SBNOR.OL",     # Sparebanken Norge
    "BFT.WA",       # Benefit Systems
    "UNIB-SDB.ST",  # Kindred Group
    "VOLCAR-B.ST",  # Volvo Car
]

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

def data_fetch(symbol, start_date, end_date, interval, name):
    df = yf.download(symbol, start=start_date, end=end_date, interval=interval, auto_adjust=False)
    df.dropna(inplace=True)
    df.to_csv(f"{symbol}_{name}.csv")

start_date = "2023-01-01"
end_date   = "2025-12-31"
interval = "1d"
for symbol in tickers:
    df_train = data_fetch(symbol, start_date, end_date, interval, name = "2024")


'\ndef data_fetch(symbol, start_date, end_date, interval, name):\n    df = yf.download(symbol, start=start_date, end=end_date, interval=interval, auto_adjust=False)\n    df.dropna(inplace=True)\n    df.to_csv(f"{symbol}_{name}.csv")\n\nstart_date = "2023-01-01"\nend_date   = "2025-12-31"\ninterval = "1d"\nfor symbol in tickers:\n    df_train = data_fetch(symbol, start_date, end_date, interval, name = "2024")'

In [2]:
import sqlite3
from datetime import datetime, timedelta

DB_PATH = "market.db"

TICKERS = [
    "AAPL", "MSFT", "GOOGL",  # ...
]

def init_db(conn: sqlite3.Connection) -> None:
    conn.execute("""
    CREATE TABLE IF NOT EXISTS prices (
      ticker     TEXT NOT NULL,
      date       TEXT NOT NULL,
      open       REAL,
      high       REAL,
      low        REAL,
      close      REAL,
      adj_close  REAL,
      volume     INTEGER,
      PRIMARY KEY (ticker, date)
    );
    """)
    conn.execute("CREATE INDEX IF NOT EXISTS idx_prices_date ON prices(date);")
    conn.commit()

def get_last_date(conn: sqlite3.Connection, ticker: str) -> str | None:
    row = conn.execute(
        "SELECT MAX(date) FROM prices WHERE ticker = ?",
        (ticker,)
    ).fetchone()
    return row[0]  # "YYYY-MM-DD" ou None

def normalize_yf(df: pd.DataFrame, ticker: str) -> pd.DataFrame:
    # yfinance -> colonnes standard: Open High Low Close Adj Close Volume
    out = df.reset_index().rename(columns={
        "Date": "date",
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Adj Close": "adj_close",
        "Volume": "volume",
    })
    out["ticker"] = ticker
    out["date"] = pd.to_datetime(out["date"]).dt.strftime("%Y-%m-%d")
    # garde uniquement les colonnes utiles
    return out[["ticker","date","open","high","low","close","adj_close","volume"]]

def upsert_prices(conn: sqlite3.Connection, df: pd.DataFrame) -> int:
    if df.empty:
        return 0

    sql = """
    INSERT INTO prices (ticker, date, open, high, low, close, adj_close, volume)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ON CONFLICT(ticker, date) DO UPDATE SET
      open      = excluded.open,
      high      = excluded.high,
      low       = excluded.low,
      close     = excluded.close,
      adj_close = excluded.adj_close,
      volume    = excluded.volume;
    """
    rows = list(df.itertuples(index=False, name=None))
    conn.executemany(sql, rows)
    conn.commit()
    return len(rows)

def download_one(ticker: str, start: str | None, end: str | None = None) -> pd.DataFrame:
    # auto_adjust=False pour garder Adj Close
    return yf.download(
        ticker,
        start=start,
        end=end,
        auto_adjust=False,
        progress=False,
        interval="1d",
        actions=False,
    )

def main():
    conn = sqlite3.connect(DB_PATH)
    init_db(conn)

    total = 0
    for t in TICKERS:
        last = get_last_date(conn, t)
        if last is None:
            start = "2000-01-01"  # ou une date plus récente si tu veux
        else:
            # on repart du lendemain (évite de recharger inutilement)
            start_dt = datetime.strptime(last, "%Y-%m-%d") + timedelta(days=1)
            start = start_dt.strftime("%Y-%m-%d")

        df = download_one(t, start=start)
        df2 = normalize_yf(df, t)
        inserted = upsert_prices(conn, df2)
        total += inserted
        print(f"{t}: last={last} start={start} -> {inserted} rows")

    conn.close()
    print(f"Done. Total rows upserted: {total}")

if __name__ == "__main__":
    main()


AAPL: last=None start=2000-01-01 -> 6539 rows
MSFT: last=None start=2000-01-01 -> 6539 rows
GOOGL: last=None start=2000-01-01 -> 5377 rows
Done. Total rows upserted: 18455


In [7]:
con = sqlite3.connect("market.db")

df = pd.read_sql_query("""
SELECT date, close
FROM prices
WHERE ticker = ?
ORDER BY date
""", con, params=("GOOGL",))

con.close()
print(df.head())


         date     close
0  2004-08-19  2.511011
1  2004-08-20  2.710460
2  2004-08-23  2.737738
3  2004-08-24  2.624374
4  2004-08-25  2.652653


In [ ]:
data = [1,2,3,4,5,6]

SyntaxError: invalid syntax (3996259930.py, line 1)